In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [3]:
scs = scs(['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'])
len(scs)

11

In [4]:
import itertools

def overlap(a, b, min_length=1):
    """Longest suffix of a matching prefix of b (>= min_length), else 0."""
    start = 0
    while True:
        start = a.find(b[:min_length], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a) - start
        start += 1

def all_scs(reads):
    """Return (min_len, set_of_all_SCS, number_of_SCS) by brute force."""
    best_len = float("inf")
    best_scs = set()

    for perm in itertools.permutations(reads):
        s = perm[0]
        for i in range(len(perm) - 1):
            o = overlap(perm[i], perm[i+1], 1)
            s += perm[i+1][o:]
        L = len(s)
        if L < best_len:
            best_len = L
            best_scs = {s}
        elif L == best_len:
            best_scs.add(s)

    return best_len, best_scs, len(best_scs)

# Example
reads = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
min_len, scs_set, count = all_scs(reads)
print("Shortest length:", min_len)
print("Number of SCS:", count)
print("All SCS:", scs_set)

Shortest length: 11
Number of SCS: 4
All SCS: {'GATTGCCTTGG', 'CCTTGGATTGC', 'TGCCTTGGATT', 'TGGATTGCCTT'}


In [5]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

--2025-12-05 04:42:37--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.240.190.188, 54.240.190.87, 54.240.190.146, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.240.190.188|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395781 (387K) [video/m2ts]
Saving to: ‘ads1_week4_reads.fq’

ads1_week4_reads.fq 100%[===================>] 386.50K  1.46MB/s    in 0.3s    

2025-12-05 04:42:38 (1.46 MB/s) - ‘ads1_week4_reads.fq’ saved [395781/395781]



In [3]:
import itertools
from collections import defaultdict

def read_fastq(filename):
    reads = []
    with open(filename) as f:
        while True:
            f.readline()
            seq = f.readline().strip()
            if not seq:
                break
            reads.append(seq)
            f.readline()
            f.readline()
    return reads

def overlap(a, b, min_length):
    """Longest suffix of a matching prefix of b (>= min_length)."""
    start = len(a) - min_length
    if start < 0:
        return 0
    while start >= 0:
        if b.startswith(a[start:]):
            return len(a) - start
        start -= 1
    return 0

def remove_contained(reads):
    reads = sorted(set(reads), key=len)
    out = []
    for i, r in enumerate(reads):
        if any(r in s for s in reads[i+1:]):
            continue
        out.append(r)
    return out

def build_prefix_index(reads, k):
    idx = defaultdict(list)
    for r in reads:
        if len(r) >= k:
            idx[r[:k]].append(r)
    return idx

def find_best_overlap(reads, k, min_overlap):
    best_a = best_b = None
    best_olen = 0
    prefix_index = build_prefix_index(reads, k)

    for a in reads:
        max_suffix_start = len(a) - min_overlap
        for start in range(max_suffix_start, -1, -1):
            suf = a[start:start + k]
            if len(suf) < k:
                continue
            for b in prefix_index.get(suf, ()):
                if a is b:
                    continue
                olen = overlap(a, b, min_overlap)
                if olen > best_olen:
                    best_a, best_b, best_olen = a, b, olen
        # small optimization: early exit if overlap reaches len(a)
        # (not necessary, but safe)
    return best_a, best_b, best_olen

def greedy_assemble_fast(reads, min_overlap=10):
    reads = remove_contained(reads)
    reads = list(reads)
    while True:
        if len(reads) == 1:
            break
        # choose k as min_overlap for the index
        a, b, olen = find_best_overlap(reads, k=min_overlap, min_overlap=min_overlap)
        if olen == 0:
            break
        merged = a + b[olen:]
        reads.remove(a)
        reads.remove(b)
        reads.append(merged)
    return "".join(reads)

def analyze_genome(genome):
    return len(genome), genome.count("A"), genome.count("T")

def assemble_and_count_fast(fq_file, min_overlap=10):
    reads = read_fastq(fq_file)
    genome = greedy_assemble_fast(reads, min_overlap=min_overlap)
    length, numA, numT = analyze_genome(genome)
    return genome, length, numA, numT

# Example:
genome, length, A, T = assemble_and_count_fast("ads1_week4_reads.fq", min_overlap=10)
print(length, A, T)

15894 4633 3723
